In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

folder = 'data/'

labels = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

print(labels)


['Lung Squamous Cell Carcinoma', 'Pancreatic Adenocarcinoma', 'Kidney Renal Clear Cell Carcinoma', 'Uveal Melanoma', 'Breast Invasive Carcinoma', 'Lung Adenocarcinoma']


In [3]:
def retrieve_data(data_path, filenames):
    list_of_dfs = []
    for i in range(0, len(filenames)):
        if ('annotations.txt' in filenames[i]):
            break
        inner_data_path = data_path + filenames[i]
        df = pd.read_csv(inner_data_path, sep='	', header=0)
        df = df[['miRNA_ID','reads_per_million_miRNA_mapped']]
        list_of_dfs.append(df)
    return list_of_dfs
    

In [12]:
def make_df(list_of_dfs, cancer_type):
    data = [] 
    labels = list_of_dfs[0]['miRNA_ID']
    for df in list_of_dfs:
        data.append(df['reads_per_million_miRNA_mapped'])

    big_df = pd.DataFrame(data)
    big_df.columns = labels
    big_df['diagnosis'] = cancer_type
    big_df.reset_index(drop=True)
    
    return big_df

In [5]:
dp = 'data/Breast Invasive Carcinoma/MANIFEST.txt'
df = pd.read_csv(dp, sep='	', header=0)
filenames = df['filename']
breast = retrieve_data('data/Breast Invasive Carcinoma/', filenames)

Add label column for each based on which folder they came from, we need to take each df and transpose it so that each df is a row with a label

In [6]:
def double_check(data_frames):
    for df in data_frames:
        if (data_frames[0]['miRNA_ID'].equals(df['miRNA_ID']) == False):
            return False
    return True

In [13]:
new = make_df(breast[:10], 'Breast Invasive Carcinoma')
new

miRNA_ID,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b,diagnosis
reads_per_million_miRNA_mapped,17225.641202,17168.697999,17165.904559,37132.340301,4217.020216,410.420821,1099.541016,4846.188888,4989.513855,655.169154,...,1.933920,0.000000,3.867840,5.586880,0.0,31.157602,47.703362,1462.043595,37550.067043,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,9675.101346,9620.924588,9710.866472,11593.826262,1817.884192,334.477994,1189.031704,4199.545275,4264.938315,511.504784,...,5.925583,0.000000,1.269768,1.058140,0.0,26.136053,44.547686,668.744359,13635.189582,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,9947.288063,10160.137808,10204.137755,9738.288314,1366.198361,243.649708,813.999023,735.899117,752.399097,195.799765,...,0.549999,0.000000,0.000000,3.849995,0.0,26.399968,24.199971,228.249726,33884.359339,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,18022.771624,18041.827151,18067.323984,17540.076324,3884.643741,230.679238,2396.299685,8376.112098,8537.815694,681.436400,...,3.891622,0.000000,1.476132,2.549683,0.0,26.033608,49.785921,1360.323117,17850.466713,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,4686.419964,4688.795641,4698.100379,2814.682994,323.884043,234.697147,1773.938243,4904.982301,5131.364576,262.215412,...,0.296960,0.000000,0.098987,0.296960,0.0,123.535234,43.554089,89.978788,28231.760414,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,10756.473826,10917.324431,10965.743745,22821.089812,4321.218657,318.418543,791.532693,4223.149028,4319.166991,487.065478,...,2.051666,0.000000,2.461999,1.641333,0.0,18.054660,45.957316,1783.307975,34241.482714,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,7888.471824,7888.020220,7970.061652,8051.500944,770.737854,1085.355454,2169.807699,3629.241975,3752.078320,695.169416,...,4.516042,0.150535,0.301069,0.602139,0.0,21.074863,104.471107,423.153146,124724.803684,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,7266.185405,7269.615728,7389.295869,25485.390638,463.855835,1107.231882,250.032397,3183.339305,3178.765541,441.749312,...,13.721290,0.000000,1.905735,3.811469,0.0,43.069605,151.315338,114.725231,9989.099197,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,11319.333225,11252.257135,11286.281238,22479.725280,3425.255120,1293.888056,2316.555400,7721.527293,7928.588267,1647.252675,...,14.581759,0.486059,0.486059,4.374528,0.0,113.737718,162.829639,755.821160,76632.002586,Breast Invasive Carcinoma
reads_per_million_miRNA_mapped,5263.441473,5422.982108,5376.799293,45701.394511,2624.023601,1305.014404,922.256828,717.233118,682.945876,214.820065,...,6.297657,0.000000,0.000000,3.498698,0.0,6.997396,16.094011,1295.218049,86638.261903,Breast Invasive Carcinoma


In [ ]:
trans = trans.drop(['miRNA_ID'])

In [ ]:
trans

In [ ]:
trans['diagnosis'] = [1]

In [ ]:
trans.reset_index(drop=True)